In [129]:
import pandas as pd
import json
import os
import gc
import csv
import time

In [130]:
def extract_information_from_CAT033(s):   
    length = int(s[2:6],16)

    information = {'Link_Technology' : '',
    'Latitude' : '',
    'Longitude' : '',
    'Pressure_Altitude' : '',
    'N_S' : '',
    'N_S_Velocity' : '',
    'E_W' : '',
    'E_W_Velocity' : '',
    'U_D' : '',
    'U_D_Velocity' : '',
    'Track_Heading_b' : '',
    'True_Magnetic_b' : '',
    'Track_Heading' : '',
    'True_Magnetic' : '',
    'Target_Identification' : '',
    'Target_Status' : '',
    'Geometric_Altitude' : ''}



    Link_Technology = Latitude = Longitude = Pressure_Altitude = N_S = N_S_Velocity = E_W = E_W_Velocity = U_D = U_D_Velocity = Track_Heading_b = True_Magnetic_b = Track_Heading = True_Magnetic = Target_Identification = Target_Status = Geometric_Altitude = ''

    def hex_to_bin(n):
        n_bin = bin(int(n, 16))[2:]
        #if len(n_bin) % 8 != 0:
        #    n_bin = '0' * (8 - len(n_bin) % 8) + n_bin
        return ('0' * (len(n) * 4 - len(n_bin)) + n_bin)


    def seperate_indicator_content(s):
        i = 0            
        while i<3:        
            if (hex_to_bin(s[6 + i * 2 : 6 + (i+1)*2])[-1] == '1') and (i < 3):
                i = i + 1
            else:
                ind =  bin(int(s[6 : 6 + (i+1)*2] , 16))[2:]
                break
        return ind, s[6 + (i+1)*2:]

    indicator, ss = seperate_indicator_content(s)

    pointer = 0

    #FRN 1 Version Number 1, ss[:2]
    if indicator[0] == '1':
        pointer = pointer + 2

    #FRN 2 Data Source Identifier 2, ss[2:6]
    if indicator[1] == '1':
        pointer = pointer + 4

    #FRN 3 Link Technology Indicator 1, ss[6:8]
    if indicator[2] == '1':
        FRN_3 = hex_to_bin(ss[pointer:pointer+2])

        pointer = pointer + 2
        if FRN_3[4] == '1':
            Link_Technology = '1090 ES'
        elif FRN_3[5] == '1':
            Link_Technology = 'UAT'
        elif FRN_3[6] == '1':
            Link_Technology = 'VDL4'
        else:
            Link_Technology = 'unknown'

    #FRN 4 Time of Applicability 3, ss[8:14]
    if indicator[3] == '1':
        pointer = pointer + 6

    #FRN 5 Target Address 4
    if indicator[4] == '1':
        FRN_5 = hex_to_bin(ss[pointer:pointer+8])
        pointer = pointer + 8

        Target_Address = hex(int(FRN_5[8:],2))[2:]

    #FRN 6 Integrity and Accuracy Parameters 2
    if indicator[5] == '1':
        pointer = pointer + 4

    #FRN 7 Latitude/Longitude 6
    if indicator[6] == '1':
        FRN_7 = hex_to_bin(ss[pointer:pointer+12])
        pointer = pointer + 12

        Lat, Lon = FRN_7[:24], FRN_7[24:]
        Latitude = str(int(Lat,2) * 360 / (2**24))
        Longitude = str(int(Lon,2) * 360 / (2**24) - 360)

    #FRN 8 Pressure Altitude 2
    if indicator[8] == '1':
        FRN_8 = hex_to_bin(ss[pointer:pointer+4])
        pointer = pointer + 4

        if FRN_8[0] == '0':
            res = 100
        elif FRN_8[0] == '1':
            res = 25

        #Pressure_Al = FRN_8[2:]
        if FRN_8[2:] == '10000000000000':
            #Pressure_Altitude = 'No pressure altitude information available'
            Pressure_Altitude = ''         
        elif FRN_8[2:][0] == '0':
            Pressure_Altitude = int(FRN_8[2:],2) * res
        elif FRN_8[2:][0] == '1':
            Pressure_Altitude = - (16384 - int(FRN_8[2:],2)) * res

        Pressure_Altitude = str(Pressure_Altitude)

    #FRN 9 Velocity (Airborne) 5
    if indicator[9] == '1':
        FRN_9 = hex_to_bin(ss[pointer:pointer+10])
        pointer = pointer + 10

        if FRN_9[2] == '0':
            so = 0.25
        elif FRN_9[2] == '1':
            so = 2

        if FRN_9[3] == '0':
            N_S = 'North'
        elif FRN_9[3] == '1':
            N_S = 'South'

        N_S_Velocity = (int(FRN_9[4:16],2) - 1) * so
        if N_S_Velocity == 0:
            N_S_Velocity = ''

        if FRN_9[16] == '0':
            E_W = 'East'
        elif FRN_9[16] == '1':
            E_W = 'West'


        E_W_Velocity = (int(FRN_9[17:29],2) - 1) * so
        if E_W_Velocity == 0:
            E_W_Velocity = ''

        if FRN_9[29] == '0':
            U_D = 'UP'
        elif FRN_9[29] == '1':
            U_D = 'DOWN'

        U_D_Velocity = (int(FRN_9[30:],2) - 1) * 32
        if U_D_Velocity == 0:
            U_D_Velocity = ''   

        N_S_Velocity = str(N_S_Velocity)
        E_W_Velocity = str(E_W_Velocity)
        U_D_Velocity = str(U_D_Velocity)


    #FRN 10 Velocity (Surface) 3
    if indicator[10] == '1':
        FRN_10 = hex_to_bin(ss[pointer:pointer+6])
        pointer = pointer + 6

        if FRN_10[0] == '0':
            Track_Heading_b = ''
            True_Magnetic_b = ''
            Track_Heading = ''
            True_Magnetic = ''

        elif FRN_10[0] =='1':

            if FRN_10[1] == '0': 
                Track_Heading_b = 'Track'
            else:
                Track_Heading_b = 'Heading'

            Track_Heading = str(int(FRN_10[3:13],2) * 360 / (2**10))

            if FRN_10[2] == '0':
                True_Magnetic_b = 'True'
            else:
                True_Magnetic_b = 'Magnetic'

            True_Magnetic = str((int(FRN_10[13:],2) - 1) * 0.125)
            if True_Magnetic == 0:
                #True_Magnetic = 'Not Available'
                True_Magnetic = ''


    #FRN 11 Mode 3/A Code 2
    if indicator[11] == '1':
        pointer = pointer + 4

    #FRN 12 Target Identification 6
    if indicator[12] == '1':
        FRN_12 = hex_to_bin(ss[pointer:pointer+12])
        pointer = pointer + 12

        dic = {
         '000000':''
        ,'000001':'A'
        ,'000010':'B'
        ,'000011':'C'
        ,'000100':'D'
        ,'000101':'E'
        ,'000110':'F'
        ,'000111':'G'
        ,'001000':'H'
        ,'001001':'I'
        ,'001010':'J'
        ,'001011':'K'
        ,'001100':'L'
        ,'001101':'M'
        ,'001110':'N'
        ,'001111':'O'
        ,'010000':'P'
        ,'010001':'Q'
        ,'010010':'R'
        ,'010011':'S'
        ,'010100':'T'
        ,'010101':'U'
        ,'010110':'V'
        ,'010111':'W'
        ,'011000':'X'
        ,'011001':'Y'
        ,'011010':'Z'
        ,'011011':''
        ,'011100':''
        ,'011101':''
        ,'011110':''
        ,'011111':''
        ,'100000':' '
        ,'100001':''
        ,'100010':''
        ,'100011':''
        ,'100100':''
        ,'100101':''
        ,'100110':''
        ,'100111':''
        ,'101000':''
        ,'101001':''
        ,'101010':''
        ,'101011':''
        ,'101100':''
        ,'101101':''
        ,'101110':''
        ,'101111':''
        ,'110000':'0'
        ,'110001':'1'
        ,'110010':'2'
        ,'110011':'3'
        ,'110100':'4'
        ,'110101':'5'
        ,'110110':'6'
        ,'110111':'7'
        ,'111000':'8'
        ,'111001':'9'
        ,'111010':''
        ,'111011':''}

        #global Target_Identification
        Target_Identification = ''

        for j in range(8):
            Target_Identification = Target_Identification + dic.get(FRN_12[j * 6: (j+1) * 6])

        Target_Identification = Target_Identification.rstrip()




    #FRN 13 Emitter Category 1
    if indicator[13] == '1':
        pointer = pointer + 2

    #FRN 14 Target Status 1
    if indicator[14] == '1':
        FRN_14 = hex_to_bin(ss[pointer:pointer+2])
        pointer = pointer + 2

        status_code_bit =   {#'0000': 'No emergency/Not reported',
                             '0000': '',
                             '0001': 'General emergency',
                             '0010': 'Lifeguard/medical emergency',
                             '0011': 'Minimum fuel',
                             '0100': 'No communications',
                             '0101': 'Unlawful interference (hijacking)',
                             '0110': 'Downed Aircraft'}
        Target_Status = ''
        Target_Status =  status_code_bit.get(FRN_14[:4])

    #FRN 15 Geometric Altitude 2
    if indicator[16] == '1':
        FRN_15 = hex_to_bin(ss[pointer:pointer+4])
        pointer = pointer + 4

        if FRN_15[0] == '0':
            Geometric_Altitude = int(FRN_15,2)  * 6.25
        elif FRN_15[0] == '1':
            Geometric_Altitude = - (65536 - int(FRN_15,2)) *  6.25
        Geometric_Altitude = str(Geometric_Altitude)


    for term in information.keys():
        if len(term) != 0 :
            information[term] = eval(term)
    
    return information

In [131]:
path = '/home/songlab/Documents/Kai/TIST2020/NEAR Lab data/adsb/2019/2019_12/'

path_processing = '/home/songlab/Documents/Kai/TIST2020/NEAR Lab data/adsb_processing/2019_12/'

ori_file = os.listdir(path)
for ori_file_name in ori_file:
    time_start = time.time()
    
    ori_directory = path + ori_file_name
    
    #print(directory)
    header_of_original = ['DayChangeFlag','RecordingDate','TimeOfApplicability','ICAOAddress','CAT033messageUUEncoded']
    file = pd.read_csv(ori_directory,names = header_of_original)
    
    header = ['DayChangeFlag','RecordingDate','TimeOfApplicability','ICAOAddress','Link_Technology','Latitude', 
          'Longitude', 'Pressure_Altitude', 'N_S', 'N_S_Velocity', 'E_W', 'E_W_Velocity', 'U_D', 'U_D_Velocity',
          'Track_Heading_b','True_Magnetic_b', 'Track_Heading', 'True_Magnetic' , 'Target_Identification',
          'Target_Status', 'Geometric_Altitude']
    df = pd.DataFrame(columns = header)
    directory_processing = path_processing + ori_file_name[:-5] + '.csv'
    df.to_csv(directory_processing,index = False)
    
    with open(directory_processing,'a+') as result:
        csv_write = csv.writer(result)
    
    
        for row in file.itertuples():#iterate by row
            dic = {'DayChangeFlag':row[1],'RecordingDate':row[2],'TimeOfApplicability':row[3],'ICAOAddress':row[4]}
            dic1 = extract_information_from_CAT033(row[5])
            dic = {**dic,**dic1}
            csv_write.writerow(dic.values())
            
    time_end = time.time()
    print(ori_file_name, time_end - time_start)    

20191213.c33t 983.0213861465454
20191208.c33t 921.0624060630798
20191209.c33t 943.7211966514587
20191221.c33t 956.0798795223236
20191217.c33t 805.4411282539368
20191226.c33t 889.8484106063843
20191204.c33t 933.7865543365479
20191227.c33t 909.2443959712982
20191214.c33t 877.20920753479
20191203.c33t 986.4944131374359
20191211.c33t 777.0684926509857
20191201.c33t 916.0891528129578
20191218.c33t 857.5307469367981
20191207.c33t 907.6908650398254
20191205.c33t 958.5966689586639
20191225.c33t 745.1674354076385
20191229.c33t 903.0628328323364
20191231.c33t 937.0906035900116
20191224.c33t 807.5059018135071
20191228.c33t 921.9310069084167
20191210.c33t 847.1644961833954
20191220.c33t 965.540070772171
20191230.c33t 882.349280834198
20191222.c33t 865.8064081668854
20191212.c33t 902.1507940292358
20191206.c33t 961.270720243454
20191223.c33t 885.5573585033417
20191202.c33t 900.7504210472107
20191216.c33t 887.4945321083069
20191219.c33t 945.3263607025146
20191215.c33t 936.219485282898


In [127]:
ori_file_name

'20170518.c33t'

In [78]:
header_of_original = ['DayChangeFlag','RecordingDate','TimeOfApplicability','ICAOAddress','CAT033messageUUEncoded']
file = pd.read_csv('/home/songlab/Documents/Kai/TIST2020/NEAR Lab data/adsb/2018/20180122.c33t',names = header_of_original)

In [97]:
header = ['DayChangeFlag','RecordingDate','TimeOfApplicability','ICAOAddress','Link_Technology','Latitude', 
          'Longitude', 'Pressure_Altitude', 'N_S', 'N_S_Velocity', 'E_W', 'E_W_Velocity', 'U_D', 'U_D_Velocity',
          'Track_Heading_b','True_Magnetic_b', 'Track_Heading', 'True_Magnetic' , 'Target_Identification',
          'Target_Status', 'Geometric_Altitude']

df = pd.DataFrame(columns = header)
df.to_csv('/home/songlab/Documents/Kai/TIST2020/NEAR Lab data/adsb_processing/2018/20180122.csv',index = False)

In [ ]:
#for row in file.itertuples():#iterate by row
#    dic = {'DayChangeFlag':row[0],'RecordingDate':row[1],'TimeOfApplicability':row[2],'ICAOAddress':row[3]}
#    dic1 = extract_information_from_CAT033(row[5])
#    dic = {**dic,**dic1}
    

In [89]:
dic

{'DayChangeFlag': 27029,
 'RecordingDate': 0,
 'TimeOfApplicability': 20180122,
 'ICAOAddress': 170022781}

In [88]:
row
#row[0]
#extract_information_from_CAT033(row[5])

Pandas(Index=87635, DayChangeFlag=0, RecordingDate=20180122, TimeOfApplicability=170835156, ICAOAddress='00a4d4b3', CAT033messageUUEncoded='210030ffcf9001b4022878899400a4d4b307d0134c55c62ad7847e02d5ac08313182b4c70820080012d8293ffb98e41a')

In [98]:
time_start = time.time()

with open('/home/songlab/Documents/Kai/TIST2020/NEAR Lab data/adsb_processing/2018/20180122.csv','a+') as result:
    csv_write = csv.writer(result)
    
    for row in file.itertuples():#iterate by row
        dic = {'DayChangeFlag':row[1],'RecordingDate':row[2],'TimeOfApplicability':row[3],'ICAOAddress':row[4]}
        dic1 = extract_information_from_CAT033(row[5])
        dic = {**dic,**dic1}
        csv_write.writerow(dic.values())
        
time_end = time.time()
print(time_end - time_start)  

363.1698181629181


In [75]:
extract_information_from_CAT033('21002affcf0001b50104773c1b00a850c30780000000000000a00000000004003b6cf51528200400f866')


{'Link_Technology': 'UAT',
 'Latitude': '0.0',
 'Longitude': '-360.0',
 'Pressure_Altitude': '',
 'N_S': 'North',
 'N_S_Velocity': '-0.25',
 'E_W': 'East',
 'E_W_Velocity': '-0.25',
 'U_D': 'DOWN',
 'U_D_Velocity': '-32',
 'Track_Heading_b': '',
 'True_Magnetic_b': '',
 'Track_Heading': '',
 'True_Magnetic': '',
 'Target_Identification': 'N635ER',
 'Target_Status': '',
 'Geometric_Altitude': ''}

In [14]:
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',120)